In [43]:
from pydub import AudioSegment
from pydub.utils import get_array_type
from plotnine import *
import numpy as np
import pandas as pd

In [2]:
filesToRead = [
    "EUDC 2015 Round 6 A.mp3"
]

In [27]:
debatesRead = []
for filePath in filesToRead:
    sound = AudioSegment.from_mp3(filePath)
    left = sound.split_to_mono()[0]
    
    # int16 as the audio was recorded with 16-bit precision. 
    numeric_array = np.fromstring(left._data, np.int16)
    debatesRead.append(numeric_array)

In [53]:
def findInterruptions(speech, threshold):
    # Will have tuples of the format (start location in array, length)
    interruptions = []
    isInterruption = False
    interruptionInstance = (0, 0)
    for i in range(0, len(speech)):
        speechAmplitude = speech[i]
        if not isInterruption:
            if abs(speechAmplitude) < threshold:
                isInterruption = True
                interruptionInstance = (i, 0)
            else:
                continue
        else:
            if abs(speechAmplitude) < threshold:
                continue
            else:
                isInterruption = False
                interruptionInstance = (interruptionInstance[0], i - interruptionInstance[0])
                interruptions.append(interruptionInstance)
                interruptionInstance = (0, 0)

In [54]:
x = findInterruptions(debatesRead[0], 30)